![](https://github.com/krondor/data-science-pot/blob/master/dsx_logo.png?raw=true)

# Introduction to Apache Spark: Querying data
This notebook guides you through querying data with Apache Spark, including how to create and use DataFrames, run SQL queries, apply functions to the results of SQL queries, join data from different data sources, and visualize data in graphs.

This notebook uses pySpark, the Python API for Spark. Some knowledge of Python is recommended. This notebook runs on Python 2 with Spark 1.6 and 2.0.

If you are new to Apache Spark, see the first module in this series: __Introduction to Apache Spark: Basic Concepts__. (need link when published)

## Table of contents

1. [Prepare the environment and the data](#getstarted)<br>
     1.1 [Enable SQL processing](#sqlprocessing)<br>
     1.2 [Download the data](#download)<br>
     1.3 [Create a DataFrame](#createdf)<br>
     1.4 [Create a table](#createtab)<br>
2. [Run SQL queries](#runsql)<br>
    2.1 [Display query results with a pandas DataFrame](#pandas)<br>
    2.2 [Run a group by query](#groupby)<br>
    2.3 [Run a subselect query](#subselect)<br>
    2.4 [Return nested JSON field values](#nested)<br>
3. [Convert RDDs to DataFrames](#convertrdd)<br>
    3.1 [Create a simple RDD](#simplerdd)<br>
    3.2 [Apply a schema](#apply)<br>
    3.3 [Create rows with named columns](#namedcol)<br>
    3.4 [Join tables](#join)<br>
4. [Create SQL functions](#sqlfuncs)<br>
5. [Convert a pandas DataFrame to a Spark DataFrame](#sparkdf)<br>
    5.1 [Get a new data set](#ufo)<br>
    5.2 [Create a pandas DataFrame](#ufopandas)<br>
    5.3 [Convert to a Spark DataFrame](#sparkufo)<br>
    5.4 [Run an SQL statement](#runufo)<br>
6. [Visualize data](#viz)<br>
    6.1 [Create a chart](#vizchart)<br>
    6.2 [Aggregate the data](#vizagg)<br>
    6.3 [Create a better chart](#vizchart2)<br>
7. [Summary and next steps](#nextsteps)

<a id="getstarted"></a>
## 1. Prepare the environment and the data

Before you can run SQL queries on data in an Apache Spark environment, you need to enable SQL processing and then move the data to the structured format of a DataFrame.

In [1]:
!pip install --user --upgrade pixiedust # fetch latest bokeh
!pip install --user --upgrade pixiedust # fetch latest version of Pixiedust

# Visualization Libraries
import brunel
import pixiedust

Requirement already up-to-date: pixiedust in /opt/conda3/lib/python3.5/site-packages (1.1.11)
Requirement already up-to-date: pixiedust in /opt/conda3/lib/python3.5/site-packages (1.1.11)
Pixiedust database opened successfully


<a id="sqlprocessing"></a>
### 1.1 Enable SQL processing
The way you enable SQL processing with Spark 1.6 is to create an SQLContext. With Spark 2.0, the preferred method is the new SparkSession object, but the SQLContext object is still supported. 

Use the predefined Spark Context, `sc`, which contains the connection information for Spark to create an SQLContext:

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
import os
from pyspark.sql import SQLContext
# Add asset from file system
df_data_1 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/BreastCancerWisconsinDataSet.csv', header='true', inferSchema = 'true')
df_data_1.show(5)

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|
+--------+---------+-----------+------------+---

In [4]:
dfdiag = df_data_1.select("id", "diagnosis")
dfdiag.show(5)

+--------+---------+
|      id|diagnosis|
+--------+---------+
|  842302|        M|
|  842517|        M|
|84300903|        M|
|84348301|        M|
|84358402|        M|
+--------+---------+
only showing top 5 rows



In [5]:
dffeatures = df_data_1.select("id","radius_mean","texture_mean","perimeter_mean","area_mean",
                              "smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean",
                              "fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se",
                              "smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se",
                              "fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst",
                              "smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst",
                              "fractal_dimension_worst")
dffeatures.show(5)

+--------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|      id|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|
+--------+-----------+------------+--------------+---------+--------

## Write Output to Project

In [ ]:
df_diag_pd = dfdiag.toPandas()
df_diag_pd.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/BreastCancerDiag.csv', sep=',', index=False)

df_features_pd = dffeatures.toPandas()
df_features_pd.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/BreastCancerFeats.csv', sep=',', index=False)

## Join

In [6]:
df_breast = dfdiag.join(dffeatures, "id", "inner")

In [7]:
display(df_breast)

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
843786,M,12.45,15.7,82.57,477.1,0.1278,0.17,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.00751,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.4,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.1244
846381,M,15.85,23.95,103.7,782.7,0.08401,0.1002,0.09938,0.05364,0.1847,0.05338,0.4033,1.078,2.903,36.58,0.009769,0.03126,0.05051,0.01992,0.02981,0.003002,16.84,27.66,112.0,876.5,0.1131,0.1924,0.2322,0.1119,0.2809,0.06287
84799002,M,14.54,27.54,96.73,658.8,0.1139,0.1595,0.1639,0.07364,0.2303,0.07077,0.37,1.033,2.879,32.55,0.005607,0.0424,0.04741,0.0109,0.01857,0.005466,17.46,37.13,124.1,943.2,0.1678,0.6577,0.7026,0.1712,0.4218,0.1341
854941,B,13.03,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.02923,0.1467,0.05863,0.1839,2.342,1.17,14.16,0.004352,0.004899,0.01343,0.01164,0.02671,0.001777,13.3,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
855625,M,19.07,24.81,128.3,1104.0,0.09081,0.219,0.2107,0.09961,0.231,0.06343,0.9811,1.666,8.83,104.9,0.006548,0.1006,0.09723,0.02638,0.05333,0.007646,24.09,33.17,177.4,1651.0,0.1247,0.7444,0.7242,0.2493,0.467,0.1038
857438,M,15.1,22.02,97.26,712.8,0.09056,0.07081,0.05253,0.03334,0.1616,0.05684,0.3105,0.8339,2.097,29.91,0.004675,0.0103,0.01603,0.009222,0.01095,0.001629,18.1,31.69,117.7,1030.0,0.1389,0.2057,0.2712,0.153,0.2675,0.07873
85759902,B,11.52,18.75,73.34,409.0,0.09524,0.05473,0.03036,0.02278,0.192,0.05907,0.3249,0.9591,2.183,23.47,0.008328,0.008722,0.01349,0.00867,0.03218,0.002386,12.84,22.47,81.81,506.2,0.1249,0.0872,0.09076,0.06316,0.3306,0.07036
858981,B,8.598,20.98,54.66,221.8,0.1243,0.08963,0.03,0.009259,0.1828,0.06757,0.3582,2.067,2.493,18.39,0.01193,0.03162,0.03,0.009259,0.03357,0.003048,9.565,27.04,62.06,273.9,0.1639,0.1698,0.09001,0.02778,0.2972,0.07712
859196,B,9.173,13.86,59.2,260.9,0.07721,0.08751,0.05988,0.0218,0.2341,0.06963,0.4098,2.265,2.608,23.52,0.008738,0.03938,0.04312,0.0156,0.04192,0.005822,10.01,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.0849
859283,M,14.78,23.94,97.4,668.3,0.1172,0.1479,0.1267,0.09029,0.1953,0.06654,0.3577,1.281,2.45,35.24,0.006703,0.0231,0.02315,0.01184,0.019,0.003224,17.31,33.39,114.6,925.1,0.1648,0.3416,0.3024,0.1614,0.3321,0.08911


In [8]:
df_breast_pd = df_breast.toPandas()
%brunel data('df_breast_pd') bar x(area_mean, diagnosis) y(#count) color(diagnosis) tooltip(#all) bin(area_mean) | bar x(diagnosis) y(area_worst) sort(#count) tooltip(#all) sum(area_worst) :: width=1100, height=400 

<IPython.core.display.Javascript object>

In [9]:
display(df_breast)

## Machine Learning

In [10]:
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
labelIndexer = StringIndexer(inputCol='diagnosis', outputCol='label').fit(df_breast)

# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["id","radius_mean","texture_mean","perimeter_mean","area_mean",
                                       "smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean",
                                       "fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se",
                                       "smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se",
                                       "fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst",
                                       "smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst",
                                       "fractal_dimension_worst"], outputCol="features")

In [11]:
# instantiate the algorithm, take the default settings
rf=RandomForestClassifier(labelCol="label", featuresCol="features")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[labelIndexer, assembler, rf, labelConverter])

In [12]:
# Split data into train and test datasets
train, test = df_breast.randomSplit([0.7,0.3], seed=6)
train.cache()
test.cache()

DataFrame[id: int, diagnosis: string, radius_mean: double, texture_mean: double, perimeter_mean: double, area_mean: double, smoothness_mean: double, compactness_mean: double, concavity_mean: double, concave points_mean: double, symmetry_mean: double, fractal_dimension_mean: double, radius_se: double, texture_se: double, perimeter_se: double, area_se: double, smoothness_se: double, compactness_se: double, concavity_se: double, concave points_se: double, symmetry_se: double, fractal_dimension_se: double, radius_worst: double, texture_worst: double, perimeter_worst: double, area_worst: double, smoothness_worst: double, compactness_worst: double, concavity_worst: double, concave points_worst: double, symmetry_worst: double, fractal_dimension_worst: double]

In [13]:
# Build models
model = pipeline.fit(train)

In [14]:
model.transform(test)

DataFrame[id: int, diagnosis: string, radius_mean: double, texture_mean: double, perimeter_mean: double, area_mean: double, smoothness_mean: double, compactness_mean: double, concavity_mean: double, concave points_mean: double, symmetry_mean: double, fractal_dimension_mean: double, radius_se: double, texture_se: double, perimeter_se: double, area_se: double, smoothness_se: double, compactness_se: double, concavity_se: double, concave points_se: double, symmetry_se: double, fractal_dimension_se: double, radius_worst: double, texture_worst: double, perimeter_worst: double, area_worst: double, smoothness_worst: double, compactness_worst: double, concavity_worst: double, concave points_worst: double, symmetry_worst: double, fractal_dimension_worst: double, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLabel: string]

In [15]:
results = model.transform(test)
results=results.select(results["id"],results["diagnosis"],results["label"],results["predictedLabel"],results["prediction"],results["probability"])
results.toPandas().head(6)

,id,diagnosis,label,predictedLabel,prediction,probability
0,8913,B,0.0,B,0.0,"[0.9406011937045566, 0.05939880629544344]"
1,85715,M,1.0,M,1.0,"[0.05769230769230769, 0.9423076923076923]"
2,86211,B,0.0,B,0.0,"[0.9943325369881386, 0.005667463011861352]"
3,86355,M,1.0,M,1.0,"[0.0, 1.0]"
4,86408,B,0.0,B,0.0,"[0.8459992036548053, 0.15400079634519467]"
5,86561,B,0.0,B,0.0,"[0.9392123048156676, 0.06078769518433232]"


In [16]:
print('Precision model = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

Precision model = 0.96.


In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

Area under ROC curve = 0.94.


In [ ]:
from dsx_ml.ml import save

model_name = "Breast_Cancer_RF"
save(name = model_name,
     model = model,
     algorithm_type = 'Classification',
     test_data = test)

In [ ]:
# Write the test data to a .csv so that we can later use it for Evaluation
writeCSV=test.toPandas()
writeCSV.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/BreastCancerModelEval.csv', sep=',', index=False)

In [ ]:
json_payload = [{
    "PHASE":"Adult",
    "ID":999,
    "GENDER":"F",
    "STATUS":"M",
    "CHILDREN":2.0,
    "ESTINCOME":77551.100000,
    "CAROWNER":"Y",
    "AGE":33,
    "LONGDISTANCE":20.530000,
    "INTERNATIONAL":0.000000,
    "LOCAL":41.890000,
    "DROPPED":1.000000,
    "PAYMETHOD":"CC",
    "LOCALBILLTYPE":"Budget",
    "LONGDISTANCEBILLTYPE":"Standard",
    "USAGE":62.420000,
    "RATEPLAN":2.000000
}]

In [ ]:
import requests, json
from pprint import pprint

scoring_endpoint = 'https://dsxl-api/v3/project/score/Python35/spark-2.2/DSX%20for%20SQL/Breast_Cancer_RF/1'

header_online = {'Content-Type': 'application/json', 'Authorization':os.environ['DSX_TOKEN']}

response_scoring = requests.post(scoring_endpoint, json=json_payload, headers=header_online)

<a id="download"></a>
### 1.2 Download the data file

You'll download a JSON file with data about world banks from GitHub.

Remove any files with the same name as the file that you're going to download and then download the file from a URL:

In [ ]:
!rm world_bank.json.gz -f
!wget https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.json.gz

<a id="createdf"></a>
### 1.3 Create a DataFrame 

Instead of creating an RDD to read the file, you'll create a Spark DataFrame. Unlike an RDD, a DataFrame creates a schema around the data, which supplies the necessary structure for SQL queries. A self-describing format like JSON is ideal for DataFrames, but many other file types are supported, including text (CSV) and Parquet.

Create a DataFrame:

In [ ]:
example1_df = sqlContext.read.json("./world_bank.json.gz")

Print the schema to see how Spark SQL inferred the shape of the data:

In [ ]:
print(example1_df.printSchema())

Now look at the first two rows of data.

You can run the simple command `print example1_df.take(2)`, however, for readability, run the following command to include a row of asterisks in between the data rows:

In [ ]:
for row in example1_df.take(2):
    print(row)
    print("*" * 20)

<a id="createtab"></a>
### 1.4 Create a table 

SQL statements must be run against a table. Create a table that's a pointer to the DataFrame:

In [ ]:
example1_df.registerTempTable("world_bank")

<a id="runsql"></a>
## 2. Run SQL queries

You must define a new DataFrame for the results of the SQL query and put the SQL statement inside the `sqlContext.sql()` method.

Run the following cell to select all columns from the table and print information about the resulting DataFrame and schema of the data:

In [ ]:
temp_df =  sqlContext.sql("select * from world_bank")

print(type(temp_df))
print("*" * 20)
print(temp_df)

The first `print` command shows that the DataFrame is a Spark DataFrame. The last `print` command shows the column names and data types of the DataFrame.

<a id="pandas"></a>
### 2.1 Display query results with a pandas DataFrame
The `print` command doesn't show the data in a useful format. Instead of creating a Spark DataFrame, use the pandas open-source data analytics library to create a pandas DataFrame that shows the data in a table. 

Import the pandas library and use the `.toPandas()` method to show the query results:

In [ ]:
import pandas as pd
sqlContext.sql("select id, borrower from world_bank limit 2").toPandas()

<a id="groupby"></a>
### 2.2 Run a group by query

You can make your SQL queries easier to read by using the `query` keyword and surrounding the SQL query with `"""` on separate lines. 

Calculate a count of projects by region:

In [ ]:
query = """
select
    regionname ,
    count(*) as project_count
from world_bank
group by regionname 
order by count(*) desc
"""

sqlContext.sql(query).toPandas()

<a id="subselect"></a>
### 2.3 Run a subselect query
You can run subselect queries.

Calculate a count of projects by region again, but this time using a subselect:

In [ ]:
query = """

select * from
    (select
        regionname ,
        count(*) as project_count
    from world_bank
    group by regionname 
    order by count(*) desc) table_alias
limit 2
"""

sqlContext.sql(query).toPandas()

<a id="nested"></a>
### 2.4 Return nested JSON field values
With JSON data, you can select the values of nested fields with dot notation.

Print the schema so that you can see that `sector.Name` is a nested field and then select its first two values:

In [ ]:
example1_df.printSchema()

sql = "select sector.Name from world_bank limit 2"
sqlContext.sql(sql).show()

<a id="convertrdd"></a>
## 3. Convert RDDs to DataFrames
If you want to run SQL queries on an existing RDD, you must convert the RDD to a DataFrame. The main difference between and RDDs and DataFrames is whether the columns are named.

You'll create an RDD and then convert it to a DataFrame in two different ways:
 - [Apply a schema](#apply)
 - [Create rows with named columns](#namedcol)

<a id="simplerdd"></a>
### 3.1 Create a simple RDD
You'll create a simple RDD with an ID column and two columns of random numbers.

First create a Python list of lists:

In [ ]:
import random

data_e2 = []
for x in range(1,6):
    random_int = int(random.random() * 10)
    data_e2.append([x, random_int, random_int^2])

Now create the RDD:

In [ ]:
rdd_example2 = sc.parallelize(data_e2)
print(rdd_example2.collect())

<a id="apply"></a>
### 3.2 Apply a schema
You'll use the `StructField` method to create a schema object that's based on a string, apply the schema to the RDD to create a DataFrame, and then create a table to run SQL queries on.

Define your schema columns as a string:

In [ ]:
from pyspark.sql.types import *

schemaString = "ID VAL1 VAL2"

Assign header information with the `StructField` method and create the schema with the `StructType` method:

In [ ]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

Apply the schema to the RDD with the `createDataFrame` method:

In [ ]:
schemaExample = sqlContext.createDataFrame(rdd_example2, schema)

Register the DataFrame as a table:

In [ ]:
schemaExample.registerTempTable("example2")

View the data:

In [ ]:
print(schemaExample.collect())

You can reference the columns names in DataFrames:

In [ ]:
for row in schemaExample.take(2):
    print(row.ID, row.VAL1, row.VAL2)

Run a simple SQL query:

In [ ]:
sqlContext.sql("select * from example2").toPandas()

<a id="namedcol"></a>
### 3.3 Create rows with named columns
You'll create an RDD with named columns and then convert it to a DataFrame and a table.

Create a new RDD and specify the names of the columns with the `map` method:

In [ ]:
from pyspark.sql import Row

rdd_example3 = rdd_example2.map(lambda x: Row(id=x[0], val1=x[1], val2=x[2]))

print(rdd_example3.collect())                                                          

Convert `rdd_example3` to a DataFrame and register an associated table:

In [ ]:
df_example3 = rdd_example3.toDF()
df_example3.registerTempTable("example3")

Run a simple SQL query:

In [ ]:
sqlContext.sql("select * from example3").toPandas()

<a id="join"></a>
### 3.4 Join tables
You can join tables.

Join tables `example2` and `example3` on the ID column:

In [ ]:
query = """
select
    *
from
    example2 e2
inner join example3 e3 on
    e2.ID = e3.id
"""

print(sqlContext.sql(query).toPandas())

Alternatively, you can join DataFrames with a Python command instead of an SQL query:

In [ ]:
df_example4 = df_example3.join(schemaExample, schemaExample["ID"] == df_example3["id"] )

for row in df_example4.take(5):
    print(row)

<a id="sqlfuncs"></a>
## 4. Create SQL functions 
You can create functions that run in SQL queries. 

First, create a Python function and test it:

In [ ]:
def simple_function(v):
    return int(v * 10)

#test the function
print(simple_function(3))

Next, register the function as an SQL function with the `registerFunction` method:

In [ ]:
sqlContext.registerFunction("simple_function", simple_function)

Now run the function in an SQL Statement:

In [ ]:
query = """
select
    ID,
    VAL1,
    VAL2,
    simple_function(VAL1) as s_VAL1,
    simple_function(VAL2) as s_VAL2
from
 example2
"""
sqlContext.sql(query).toPandas()

The values in the VAL1 and VAL2 columns look like strings (10 characters instead of a number multiplied by 10). That's because string is the default data type for columns in Spark DataFrames.

Cast the values in the VAL1 and VAL2 columns to integers: 

In [ ]:
query = """
select
    ID,
    VAL1,
    VAL2,
    simple_function(cast(VAL1 as int)) as s_VAL1,
    simple_function(cast(VAL2 as int)) as s_VAL2
from
 example2
"""
sqlContext.sql(query).toPandas()

That looks better!

<a id="sparkdf"></a>
## 5. Convert a pandas DataFrame to a Spark DataFrame
Although pandas DataFrames display data in a friendlier format, Spark DataFrames can be faster and more scalable.

You'll get a new data set, create a pandas DataFrame for it, and then convert the pandas DataFrame to a Spark DataFrame.

<a id="ufo"></a>
### 5.1 Get a new data set
Get a data set about UFO sightings:

In [ ]:
!rm SIGHTINGS.csv -f
!wget https://ibm.box.com/shared/static/krxz5fo4ojequwb45rcb9lt8fxfcm1vm.csv -O SIGHTINGS.csv

<a id="ufopandas"></a>
### 5.2 Create a pandas DataFrame
Create a pandas DataFrame of the data set with the `read_csv` method:

In [ ]:
# Transform Date Format Lambda from Source CSV
dateHandle = lambda x: pd.datetime.strptime(x, '%m/%d/%y')

pandas_df = pd.read_csv("./SIGHTINGS.csv", parse_dates=['Reports'], date_parser=dateHandle, encoding="utf-8-sig")

# Handle Dates as String to Simplify Spark Example
pandas_df['Reports'] = pandas_df['Reports'].astype(str)
pandas_df.head()

<a id="sparkufo"></a>
### 5.3 Convert to a Spark DataFrame
Convert the pandas DataFrame to a Spark DataFrame with the `createDataFrame` method. Remember using the `createDataFrame` method to convert an RDD to a Spark DataFrame?

In [ ]:
spark_df = sqlContext.createDataFrame(pandas_df)

Print the first two rows:

In [ ]:
for row in spark_df.take(2):
    print(row)


Register the Spark DataFrame as a table:

In [ ]:
spark_df.registerTempTable("ufo_sightings")

<a id="runufo"></a>
### 5.4 Run an SQL statement
Now run an SQL statement to print the first 10 rows of the table:

In [ ]:
print(sqlContext.sql("select * from ufo_sightings limit 10").collect())

<a id="viz"></a>
## 6. Visualize data
It's easy to create charts from pandas DataFrames. You'll use the matplotlib library to create graphs and the NumPy package for computing.

Import the libraries and specify to show graphs inline:

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt, numpy as np

Convert the Spark DataFrame with UFO data to a pandas DataFrame: 

In [ ]:
ufos_df = spark_df.toPandas()

<a id="vizchart"></a>
### 6.1 Create a chart

To create a chart, you call the `plot()` method and specify the type of chart, the columns for the X and Y axes, and, optionally, the size of the chart. 

For more information about plotting pandas DataFrames, see [Visualization](http://pandas.pydata.org/pandas-docs/stable/visualization.html).

Create a bar chart 12" wide by 5" high that shows the number of reports by date:

In [ ]:
ufos_df.plot(kind='bar', x='Reports', y='Count', figsize=(12, 5))

This chart doesn't look good because there are too many observations. Check how many observations there are by querying the `ufo_sightings` table:

In [ ]:
print(sqlContext.sql("select count(*) from ufo_sightings").collect())

<a id="vizagg"></a>
### 6.2 Aggregate the data

To reduce the number of data points on the chart, you can aggregate the data by year. Here are a few of the ways that you can do that:

 - Run an SQL query the Reports column to output the year and then run a group by operation on the year.
 - Create a simple Python function to aggregate by year and then run the function in an SQL query.
 - Run the `map()` method on the Spark Dataframe to append a new column that contains the aggregated count by year. This is the method you'll use.

Remember that the dates in the Reports column look like this: 2016-01-31. Therefore, to create the year column, you just need the first 4 characters from the Reports column.

Add a year column to the DataFrame:

In [ ]:
#ufos_df = spark_df.map(lambda x: Row(**dict(x.asDict(), year=int(x.Reports[0:4])))).toDF()
# new version
spark_df.head()
ufos_df = spark_df.rdd.map(lambda x: Row(**dict(x.asDict(), year=int(x.Reports[0:4])))).toDF()

Check to verify that you get the expected results:

In [ ]:
print(ufos_df.take(5))

Register the DataFrame as a table:

In [ ]:
ufos_df.registerTempTable("ufo_withyear")

<a id="vizchart2"></a>
### 6.3 Create a better chart

Now run an SQL query to group by year, order by year, and filter to the last 66 years. Then create a pandas DataFrame for the results and create a chart of the number of reports by year.

In [ ]:
query = """
select 
    sum(Count) as count, 
    year 
from ufo_withyear
where year > 1950
group by year
order by year
"""
pandas_ufos_withyears = sqlContext.sql(query).toPandas()
pandas_ufos_withyears.plot(kind='bar', x='year', y='count', figsize=(12, 5))

Now you have a chart that you can read. Look back at the original chart and notice that it wasn't ordered ascending by year.

<a id="nextsteps"></a>
## 7. Summary and next steps
You've learned how to create DataFrames, convert between DataFrame types, and convert from RDDs. You know how to run SQL queries and create SQL functions. And you can visualize the data in charts. 

Dig deeper:
 - [Apache Spark documentation](http://spark.apache.org/documentation.html)
 - [PySpark documentation](http://spark.apache.org/docs/latest/api/python/pyspark.html)
 - [pandas](http://pandas.pydata.org/pandas-docs/stable/index.html)
 - [matplotlib](http://matplotlib.org/)
 - [NumPy](http://www.numpy.org/)

### Authors
